In [59]:
import json
import csv
import pandas as pd

In [60]:
json_data = open("mpd_1.json")
jdata = json.load(json_data)
final_list = [each["_source"] for each in jdata["hits"]["hits"]]
event_sample = pd.DataFrame(final_list)

In [61]:
event_sample.head()

,Logged_In,browser,browser_version,city,device,distinct_id,event,initial_referrer,item_names,item_prices,parent_id,price,referrer,time,utm_campaign,utm_medium,utm_source
0,NaN,Chrome,60.0,Thane,NaN,15e46747739237-0d8683980719c5-3a3e5f04-100200-...,Order Placed Web,https://www.google.co.in/,"[Golden Crush Silk Unstitched Safa, Golden Vel...","[539, 599]",NaN,NaN,https://flyrobe.com/men/safa-and-stoles,1505035976,NaN,NaN,NaN
1,NaN,Chrome,60.0,Delhi,NaN,10202258670263829,Order Placed Web,https://www.google.co.in/,[Pastel Green Gota Patti work Lehenga Set],[1999],NaN,NaN,https://flyrobe.com/women/lehengas,1505047382,NaN,NaN,NaN
2,NaN,Chrome,60.0,Bengaluru,NaN,15e6c19575a16b-079e6a7d307e7b-546b3971-15f900-...,Order Placed Web,$direct,[Slim Fit Black Single-Breasted Two-Piece Suit],[1499],NaN,NaN,NaN,1505072400,NaN,NaN,NaN
3,NaN,Chrome,60.0,Mumbai,NaN,shrshtmishra@gmail.com,Order Placed Web,https://www.google.co.in/,[Tea Green Georgette Lehenga with Drape],[1499],NaN,NaN,https://flyrobe.com/women/saree-gowns,1504955110,NaN,NaN,NaN
4,NaN,Chrome,60.0,Mumbai,NaN,15e52f715326bd-02a19f787a3e76-e313761-100200-1...,Order Placed Web,https://www.google.co.in/,[Blue Embroidered Crop Top With Red Skirt],[1499],NaN,NaN,http://flyrobe.com/women/ethnic-crop-top-skirt...,1504962758,NaN,NaN,NaN


In [62]:
user_event_timeseries = event_sample[['event','distinct_id', 'time', 'parent_id']]
#dir(user_event_timeseries)
#user_event_timeseries.iteritems

In [64]:
final_list = []
user_dict = {}
corr_matrix_inv = {}
corr_matrix_brand = {}
corr_matrix_pt = {}
record_inv_detail = {}
record_inv_atc = {}

#user_event_timeseries = event_sample[['event','distinct_id', 'time', 'parent_id']]

#creating list of look_up sheet

# print(jdata["hits"]["hits"])


# Creating user-event timeseries
#import pdb;pdb.set_trace()

# for each in user_event_timeseries.iteritems():
#     print(each, "\n")
# list1 = list()
for index, each in user_event_timeseries.iterrows():
    if each["event"] == "Details Web" or each["event"] == "Add To Cart Web":
        if each["distinct_id"] not in user_dict:
            user_dict[each["distinct_id"]] = list1
        user_dict[each["distinct_id"]] = user_dict[each["distinct_id"]] + [{each["event"]: [each["time"], each["parent_id"]]}]

#print(user_dict)
# Extraction of First Detail View and corresponding Add To Cart
for user in user_dict:
    record_inv_detail[user] = []
    record_inv_atc[user] = []
    for event in user_dict[user]:
        for key, value in event.items():
            if key == "Details Web":
                record_inv_detail[user].append(value)
            if key == "Add To Cart Web":
                record_inv_detail[user].append(value)
                if (record_inv_detail[user][-1][0] - record_inv_detail[user][0][0] < 2000):
                    record_inv_atc[user].append([record_inv_detail[user][0], record_inv_detail[user][-1]])
                    record_inv_detail[user].clear()
                else:
                    record_inv_detail[user].clear()

# List of [FDP, ATC]
#record_inv_atc = {"1" : [[[1234,"FRES1"],[1235, "FRES8"]],[[3455, "FRES6"],[3675, "FRES9"]],[[2222, "FRES45"],[6583, "FRES8"]],[[8763, "FRES2"],[7757, "FRES12"]]]}
fdp_atc = []
for user, list in record_inv_atc.items():
    for cycle in list:
        list_1 = []
        for entry in cycle:
            list_1 += [entry[1]]
        fdp_atc.append(list_1)
print(fdp_atc)

# List of inventory_category : [inv_1,inv_2]


# with open('look_up.csv', 'r') as f:
#     reader = csv.reader(f)
    
#     look_up = list(reader)


with open('look_up.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        look_up.append(row)
print(look_up)

inv_cat_list = []
for element in fdp_atc:
    list_2 = []
    for each in element:
        #get inv_cat for the item : inv_cat_item
        for item in look_up:
            if item[1] == each:
                inv_cat_item = item[6]
        list_2 += [inv_cat_item]
    inv_cat_list.append(list_2)

# Creating a dict of FDP as key and list of ATC as value
inv_cat_dict = {}
for each in inv_cat_list:
    if each[0] not in inv_cat_dict:
        inv_cat_dict[each[0]] = [each[1]]
    else:
        inv_cat_dict[each[0]] += [each[1]]
print(inv_cat_dict)
# Creation of Correlation matrix of inv_cat


# Similarly for Brand
# List of brand_id : [inv_1,inv_2]
# brand_id_list = []
# for element in fdp_atc:
#     list_2 = []
#     for each in element:
#         #get brand_id for the item : brand_id_item
#         for item in look_up:
#             if item[1] == each:
#                 brand_id_item = item[4]
#         list_2 += [brand_id_item]
#     brand_id_list.append(list_2)
#
# # Creating a dict of FDP as key and list of ATC as value
# brand_id_dict = {}
# for each in brand_id_list:
#     if each[0] not in brand_id_dict:
#         brand_id_dict[each[0]] = [each[1]]
#     else:
#         brand_id_dict[each[0]] += [each[1]]
#
# # Correlation matrix of brand_id
#
#
# # Similarly for Color
# # List of inventory_category : [inv_1,inv_2]
# color_list = []
# for element in fdp_atc:
#     list_2 = []
#     for each in element:
#         #get color for the item : color_item
#         for item in look_up:
#             if item[1] == each:
#                 color_item = item[5]
#         list_2 += [color_item]
#     colort_list.append(list_2)
#
# # Creating a dict of FDP as key and list of ATC as value
# color_dict = {}
# for each in color_list:
#     if each[0] not in color_dict:
#         color_dict[each[0]] = [each[1]]
#     else:
#         color_dict[each[0]] += [each[1]]
#
# # Correlation matrix for color

[]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

